# Introduction to PydanticAI.
This module demonstrates how PydanticAI makes it easier to build production-grade LLM-powered systems with type safety and structured responses.

In [3]:
from typing import Dict, List, Optional
from utils.markdown import to_markdown

#Only for Jupyter Notebooks
import nest_asyncio
nest_asyncio.apply()

# Use Ollama locally

In [4]:
from pydantic import BaseModel, Field
from pydantic_ai import Agent, ModelRetry, RunContext, Tool

from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
 

provider = OpenAIProvider(
    base_url="http://localhost:11434/v1/"
    )
 
model = OpenAIModel(
    model_name="llama3.1:8b",
    provider=provider,   
    )


# 1. Simple Agent - Hello World Example
This example demonstrates the basic usage of PydanticAI agents.
Key concepts:
- Creating a basic agent with a system prompt
- Running synchronous queries
- Accessing response data, message history, and costs

In [5]:
agent1 = Agent(
    model=model,
    system_prompt="You are a helpful customer support agent. Be concise and friendly.",
)

# Example usage of basic agent
response = agent1.run_sync(
    user_prompt="How can I track my order #12345?"
    )

print("output:  {response.output}")
print(response.new_messages())
print(response.all_messages())

usage_data = response.usage()
if usage_data:
    print("\n--- Usage ---")
    print(f"Total Requests: {usage_data.requests}")
    print(f"Input Tokens: {usage_data.input_tokens}")
    print(f"Output Tokens: {usage_data.output_tokens}")
    print(f"Total Tokens: {usage_data.total_tokens}")
else:
    print("Usage data not available.")


output:  {response.output}
[ModelRequest(parts=[SystemPromptPart(content='You are a helpful customer support agent. Be concise and friendly.', timestamp=datetime.datetime(2025, 8, 29, 8, 37, 21, 884310, tzinfo=datetime.timezone.utc)), UserPromptPart(content='How can I track my order #12345?', timestamp=datetime.datetime(2025, 8, 29, 8, 37, 21, 884315, tzinfo=datetime.timezone.utc))]), ModelResponse(parts=[TextPart(content='To check the status of your order, please visit our website at [website URL] and click on "Track Order" in the top right corner. Alternatively, you can reply with your email address associated with the order, and we can send you an update on its current status!')], usage=RequestUsage(input_tokens=38, output_tokens=57), model_name='llama3.1:8b', timestamp=datetime.datetime(2025, 8, 29, 8, 37, 24, tzinfo=TzInfo(UTC)), provider_request_id='chatcmpl-253')]
[ModelRequest(parts=[SystemPromptPart(content='You are a helpful customer support agent. Be concise and friendly.', 

In [7]:
response2 = agent1.run_sync(
    user_prompt="What was my previous question?",
    message_history=response.new_messages(),
)
print(response2.output)
print(response.new_messages())
print(response.all_messages())

Your previous question was how to track your order #12345.
[ModelRequest(parts=[SystemPromptPart(content='You are a helpful customer support agent. Be concise and friendly.', timestamp=datetime.datetime(2025, 8, 28, 15, 1, 34, 962626, tzinfo=datetime.timezone.utc)), UserPromptPart(content='How can I track my order #12345?', timestamp=datetime.datetime(2025, 8, 28, 15, 1, 34, 962629, tzinfo=datetime.timezone.utc))]), ModelResponse(parts=[TextPart(content='To track your order #12345, you can follow these steps:\n\n1. Visit our website and click on "Track Your Order" at the top right corner.\n2. Enter your order number (12345) in the required field.\n3. Click "Track Now".\n\nAlternatively, you can also reply to this chat with the order number and I\'d be happy to check its status for you.\n\nWhich method would you prefer?')], usage=RequestUsage(input_tokens=38, output_tokens=87), model_name='llama3.1:8b', timestamp=datetime.datetime(2025, 8, 28, 15, 1, 37, tzinfo=TzInfo(UTC)), provider_re

# 2. Agent with Structured Response
This example shows how to get structured, type-safe responses from the agent. Key concepts:
- Using Pydantic models to define response structure
- Type validation and safety
- Field descriptions for better model understanding

In [19]:
class ResponseModel(BaseModel):
    """Structured response with metadata."""
    response: str
    needs_escalation: bool
    follow_up_required: bool
    sentiment: str = Field(description="Customer sentiment analysis")


agent2 = Agent(
    model=model,
    output_type=ResponseModel,
    system_prompt=(
        "You are an intelligent customer support agent. "
        "Analyze queries carefully and provide structured responses."
    ),
)

response = agent2.run_sync(
    user_prompt="How can I track my order #12345?"
    )

print(response.output.model_dump_json(indent=2))


{
  "response": "To check the status of your order #12345, please visit our website and use the tracking ID provided in your confirmation email. If you're unable to locate this information, please reply to this email with your order number and we'll be happy to assist further.",
  "needs_escalation": false,
  "follow_up_required": true,
  "sentiment": "neutral"
}


# 3. Agent with Structured Response & Dependencies

This example demonstrates how to use dependencies and context in agents.
Key concepts:
- Defining complex data models with Pydantic
- Injecting runtime dependencies
- Using dynamic system prompts


In [ ]:
# Define order schema
class Order(BaseModel):
    """Structure for order details."""
    order_id: str
    status: str
    items: List[str]


# Define customer schema
class CustomerDetails(BaseModel):
    """Structure for incoming customer queries."""
    customer_id: str
    name: str
    email: str
    orders: Optional[List[Order]] = None


# Agent with structured output and dependencies
agent3 = Agent(
    model=model,
    output_type=ResponseModel,
    deps_type=CustomerDetails,
    retries=3,
    system_prompt=(
        "You are an intelligent customer support agent. "
        "Analyze queries carefully and provide structured responses. "
        "Always great the customer and provide a helpful response."
    ),  # These are known when writing the code
)


# Add dynamic system prompt based on dependencies
@agent3.system_prompt
async def add_customer_name(ctx: RunContext[CustomerDetails]) -> str:
    return f"Customer details: {to_markdown(ctx.deps)}"  # These depend in some way on context that isn't known until runtime


customer = CustomerDetails(
    customer_id="1",
    name="John Doe",
    email="john.doe@example.com",
    orders=[
        Order(order_id="12345", status="shipped", items=["Blue Jeans", "T-Shirt"]),
    ],
)

response = agent3.run_sync(user_prompt="What did I order?", deps=customer)
print(response.all_messages())

print(f"\nmodel_dump_json:\n", response.output.model_dump_json(indent=2))

print(f"\n"
    "Customer Details:\n"
    f"Name: {customer.name}\n"
    f"Email: {customer.email}\n\n"
    "Response Details:\n"
    f"{response.output.response}\n\n"
    "Status:\n"
    f"Follow-up Required: {response.output.follow_up_required}\n"
    f"Needs Escalation: {response.output.needs_escalation}"
)


[ModelRequest(parts=[SystemPromptPart(content='You are an intelligent customer support agent. Analyze queries carefully and provide structured responses. Always great the customer and provide a helpful response.', timestamp=datetime.datetime(2025, 8, 26, 12, 19, 27, 453050, tzinfo=datetime.timezone.utc)), SystemPromptPart(content='Customer details: ## CUSTOMER_ID\n1\n\n## NAME\nJohn Doe\n\n## EMAIL\njohn.doe@example.com\n\n## ORDERS\n### ORDER_ID\n12345\n\n### STATUS\nshipped\n\n### ITEMS\n- Blue Jeans\n- T-Shirt\n\n\n', timestamp=datetime.datetime(2025, 8, 26, 12, 19, 27, 453105, tzinfo=datetime.timezone.utc)), UserPromptPart(content='What did I order?', timestamp=datetime.datetime(2025, 8, 26, 12, 19, 27, 453110, tzinfo=datetime.timezone.utc))]), ModelResponse(parts=[TextPart(content='{"name": "final_result", "parameters": {\n"follow_up_required": False,\n"needs_escalation": False,\n"response": "You have ordered a Blue Jeans and T-Shirt. Your order number is 12345, and the status is 

# 4. Agent with Tools
This example shows how to enhance agents with custom tools.
Key concepts:
- Creating and registering tools
- Accessing context in tools


In [122]:
shipping_info_db: Dict[str, str] = {
    "12345": "Shipped on 2024-12-01",
    "67890": "Out for delivery",
}

customerDetail4 = CustomerDetails(
    customer_id="1",
    name="John Doe",
    email="john.doe@example.com",
    orders=[
        Order(order_id="12345", status="shipped", items=["Blue Jeans", "T-Shirt"]),
    ],
)

def get_shipping_info(ctx: RunContext[CustomerDetails]) -> str:
    """Get the customer's shipping information."""
    return shipping_info_db[ctx.deps.orders[0].order_id]


# Agent with structured output and dependencies
agent4 = Agent(
    model=model,
    output_type=ResponseModel,
    deps_type=CustomerDetails,
    retries=3,    
    system_prompt=(
        "You are an intelligent customer support agent. "
        "Analyze queries carefully and provide structured responses. "
        "Use tools to look up relevant information."
        "Always great the customer and provide a helpful response."
    ),  # These are known when writing the code
    
    # Add tool via kwarg
    tools=[Tool(get_shipping_info, takes_ctx=True)],  
)


@agent4.system_prompt
async def add_customer_name(ctx: RunContext[CustomerDetails]) -> str:
    return f"Customer details: {to_markdown(ctx.deps)}"


response = agent4.run_sync(
    #user_prompt="What's the ship status and ship date of my last order?",
    user_prompt="What's the status of my last order?", 
    deps=customerDetail4
    )

print(response.all_messages())
print(response.output.model_dump_json(indent=2))

print(
    f"\nCustomer Details:\n"
    f"Name: {customer.name}\n"
    f"Email: {customer.email}\n\n"
    "Response Details:\n"
    f"{response.output.response}\n\n"
    "Status:\n"
    f"Follow-up Required: {response.output.follow_up_required}\n"
    f"Needs Escalation: {response.output.needs_escalation}"
)


[ModelRequest(parts=[SystemPromptPart(content='You are an intelligent customer support agent. Analyze queries carefully and provide structured responses. Use tools to look up relevant information.Always great the customer and provide a helpful response.', timestamp=datetime.datetime(2025, 8, 26, 15, 9, 40, 972785, tzinfo=datetime.timezone.utc)), SystemPromptPart(content='Customer details: ## CUSTOMER_ID\n1\n\n## NAME\nJohn Doe\n\n## EMAIL\njohn.doe@example.com\n\n## ORDERS\n### ORDER_ID\n12345\n\n### STATUS\nshipped\n\n### ITEMS\n- Blue Jeans\n- T-Shirt\n\n\n', timestamp=datetime.datetime(2025, 8, 26, 15, 9, 40, 972825, tzinfo=datetime.timezone.utc)), UserPromptPart(content="What's the status of my last order?", timestamp=datetime.datetime(2025, 8, 26, 15, 9, 40, 972827, tzinfo=datetime.timezone.utc))]), ModelResponse(parts=[TextPart(content='{"name": "get_order_info", "parameters": {"order_id": "12345}} \n\n(Note: The function is assumed to exist, but it\'s not present in provided fun

# 5. Agent with Reflection and Self-Correction
This example demonstrates advanced agent capabilities with self-correction.
Key concepts:
- Implementing self-reflection
- Handling errors gracefully with retries
- Using ModelRetry for automatic retries
- Decorator-based tool registration

In [ ]:
# Simulated database of shipping information
shipping_info_db: Dict[str, str] = {
    "#12345": "Shipped on 2024-12-01",
    "#67890": "Out for delivery",
}


customerDetail5 = CustomerDetails(
    customer_id="1",
    name="John Doe",
    email="john.doe@example.com",
    orders=[
        Order(order_id="#12345", status="shipped", items=["Blue Jeans", "T-Shirt"]),
        Order(order_id="#67890", status="Not shipped", items=["T-Shirt"]),
    ],
)

agent5 = Agent(
    model=model,
    model_settings={"temperature": 0.1}, 
    output_type=ResponseModel,
    deps_type=CustomerDetails,
    retries=3,
    system_prompt=(
        "You are an intelligent customer support agent. Analyze queries carefully and provide structured responses. "
        "Use tools to look up relevant information. Always greet the customer and provide a helpful response."
    ),
)


@agent5.tool_plain()
def get_shipping_status(order_id: str) -> str:
    """Get the shipping status for a given order ID."""
    shipping_status = shipping_info_db.get(order_id)
    if shipping_status is None:
        raise ModelRetry(
            f"No shipping information found for order ID {order_id}. Make sure the order ID starts with a #: e.g, #624743 "
            "Self-correct this if needed and try again."
        )
    return shipping_info_db[order_id]


@agent5.system_prompt
async def add_customer_name(ctx: RunContext[CustomerDetails]) -> str:
    return f"Customer details: {to_markdown(ctx.deps)}"


# Example usage
response = agent5.run_sync(
    user_prompt="What's the ship status and ship date of my order #12345 ?", 
    deps=customerDetail5
    )


print(response.output.model_dump_json(indent=3))


{
   "response": "The shipping status for your order #12345 is shipped, and the ship date was December 1st, 2024.",
   "needs_escalation": false,
   "follow_up_required": true,
   "sentiment": "neutral"
}
